In [ ]:
!pip install datasketch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 1.4 MB/s eta 0:00:00


In [ ]:
!pip install gensim

In [ ]:
!pip install pot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 7.5 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
from hashlib import sha1
from datasketch.minhash import MinHash
from datasketch.weighted_minhash import WeightedMinHashGenerator
from datasketch.lsh import MinHashLSH

import pandas as pd
import re

from nltk.lm import Vocabulary
from nltk.util import ngrams, bigrams

import gensim.downloader as gs_downloader

import faiss

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy import sparse
import numpy as np

import seaborn as sns

import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = 'drive/MyDrive/project_data/items.csv'

In [ ]:
names_df = pd.read_csv(
    path
    )[['item_id', 'title','title_orig', 'release_year']]
names_df[['title','title_orig']] = names_df[['title','title_orig']].fillna(' ')

names_df['release_year'] = (
    names_df['release_year']
    .fillna(0)
    .apply(lambda x: str(int(x)))
)

names_df['search_txt'] = (
    names_df[['title']]
    .apply(' '.join, axis=1)
    .apply(lambda x: re.sub(r'\W+', '', x).lower())
)

In [ ]:
names_df['title'][5585:5607]

5585                                 Лунтик и его друзья
5586    Закон и порядок: Отдел оперативных расследований
5587                                          Круговорот
5588                         Азбука-малышка тётушки Совы
5589                                     Хороший человек
5590                       Девушка в реке: Цена прощения
5591                            Vice: Укротители кризиса
5592                          Слова, что создали Америку
5593                                      Бульвар страха
5594                                    Полярная история
5595                  Мужской сезон: Бархатная революция
5596                                        День свадьбы
5597                      Красные туфельки и семь гномов
5598                              Плохой хороший человек
5599                            Как пишется слово Солнце
5600                                      Роковая ошибка
5601                                      Голова Горгоны
5602                           

In [ ]:
names_df[['item_id', 'title']].sample(n=50, random_state=42).to_excel('sample_to_markup.xlsx', index=False)

In [ ]:
assessment_df = pd.read_excel('/content/drive/MyDrive/project_data/sample_to_markup.xlsx')
assessment_df.head()

,item_id,title,query
0,10021,История искусств вместе с Хрюшей и...,история искусств
1,10021,История искусств вместе с Хрюшей и...,история искусств вместе
2,10021,История искусств вместе с Хрюшей и...,история искусств c Хрюшей
3,10021,История искусств вместе с Хрюшей и...,история искуств c хрюший
4,14274,Playmobil Фильм: Через вселенные,playmobil


In [ ]:
names_df

,item_id,title,title_orig,release_year,search_txt
0,10711,Поговори с ней,Hable con ella,2002,поговорисней
1,2508,Голые перцы,Search Party,2014,голыеперцы
2,10716,Тактическая сила,Tactical Force,2011,тактическаясила
3,7868,45 лет,45 Years,2015,45лет
4,16268,Все решает мгновение,,1978,всерешаетмгновение
...,...,...,...,...,...
15958,6443,Полярный круг,Arctic Circle,2018,полярныйкруг
15959,2367,Надежда,,2020,надежда
15960,10632,Сговор,Hassel,2017,сговор
15961,4538,Среди камней,Darklands,2019,средикамней


# 1. Модель на ngrams + CountVectorizer

In [ ]:
test_name = names_df.iloc[0, 4]
test_name

'поговорисней'

In [ ]:
print(list(ngrams(test_name, 1)))
print(list(bigrams(test_name)))
print(list(ngrams(test_name, 3)))
print(list(ngrams(test_name, 4)))

[('п',), ('о',), ('г',), ('о',), ('в',), ('о',), ('р',), ('и',), ('с',), ('н',), ('е',), ('й',)]
[('п', 'о'), ('о', 'г'), ('г', 'о'), ('о', 'в'), ('в', 'о'), ('о', 'р'), ('р', 'и'), ('и', 'с'), ('с', 'н'), ('н', 'е'), ('е', 'й')]
[('п', 'о', 'г'), ('о', 'г', 'о'), ('г', 'о', 'в'), ('о', 'в', 'о'), ('в', 'о', 'р'), ('о', 'р', 'и'), ('р', 'и', 'с'), ('и', 'с', 'н'), ('с', 'н', 'е'), ('н', 'е', 'й')]
[('п', 'о', 'г', 'о'), ('о', 'г', 'о', 'в'), ('г', 'о', 'в', 'о'), ('о', 'в', 'о', 'р'), ('в', 'о', 'р', 'и'), ('о', 'р', 'и', 'с'), ('р', 'и', 'с', 'н'), ('и', 'с', 'н', 'е'), ('с', 'н', 'е', 'й')]


In [ ]:
names_df.iloc[:5, :].to_dict()

{'item_id': {0: 10711, 1: 2508, 2: 10716, 3: 7868, 4: 16268},
 'title': {0: 'Поговори с ней',
  1: 'Голые перцы',
  2: 'Тактическая сила',
  3: '45 лет',
  4: 'Все решает мгновение'},
 'title_orig': {0: 'Hable con ella',
  1: 'Search Party',
  2: 'Tactical Force',
  3: '45 Years',
  4: ' '},
 'release_year': {0: '2002', 1: '2014', 2: '2011', 3: '2015', 4: '1978'},
 'search_txt': {0: 'поговорисней',
  1: 'голыеперцы',
  2: 'тактическаясила',
  3: '45лет',
  4: 'всерешаетмгновение'}}

In [ ]:
names_df['token'] = names_df['search_txt'].apply(
    lambda x: ' '.join(list(map(''.join, list(ngrams(x, 3)))))
)

In [ ]:
tokens_bi_lst = names_df['token'].to_list()

In [ ]:
tokens_bi_lst[:5]

['пог ого гов ово вор ори рис исн сне ней',
 'гол олы лые ыеп епе пер ерц рцы',
 'так акт кти тич иче чес еск ска кая аяс яси сил ила',
 '45л 5ле лет',
 'все сер ере реш еша шае ает етм тмг мгн гно нов ове вен ени ние']

In [ ]:
vectorizer_cnt = CountVectorizer(
    encoding='utf-8',
    decode_error='strict',
    lowercase=True
    )

In [ ]:
vect_bigram_df = vectorizer_cnt.fit_transform(
        tokens_bi_lst
    )

In [ ]:
len(vectorizer_cnt.get_feature_names_out())

13387

In [ ]:
vectorizer_cnt.get_feature_names_out()[:20]

array(['000', '002', '003', '004', '005', '006', '007', '009', '00в',
       '00д', '00к', '00л', '00м', '00р', '00с', '00ф', '010', '011',
       '012', '013'], dtype=object)

In [ ]:
vectorizer_cnt.get_feature_names_out()[-100:]

array(['ёде', 'ёдн', 'ёдо', 'ёдр', 'ёещ', 'ёже', 'ёжи', 'ёжк', 'ёжн',
       'ёжь', 'ёзд', 'ёзк', 'ёзл', 'ёзн', 'ёзо', 'ёзы', 'ёил', 'ёим',
       'ёис', 'ёк1', 'ёкр', 'ёли', 'ёлк', 'ёло', 'ёлт', 'ёлы', 'ёлю',
       'ёма', 'ёмв', 'ёмг', 'ёме', 'ёмк', 'ёмм', 'ёмн', 'ёмо', 'ёмр',
       'ёмс', 'ёна', 'ёне', 'ёнк', 'ёнл', 'ённ', 'ёно', 'ёну', 'ёны',
       'ёог', 'ёод', 'ёое', 'ёом', 'ёор', 'ёпа', 'ёпо', 'ёпр', 'ёра',
       'ёрд', 'ёрз', 'ёри', 'ёрк', 'ёрл', 'ёрн', 'ёро', 'ёрс', 'ёрт',
       'ёрф', 'ёрш', 'ёса', 'ёсе', 'ёси', 'ёсл', 'ёсо', 'ёст', 'ёсы',
       'ёта', 'ётв', 'ётд', 'ёте', 'ётз', 'ётм', 'ётн', 'ёто', 'ётр',
       'ётс', 'ётт', 'ёту', 'ётч', 'ёты', 'ётя', 'ёхм', 'ёху', 'ёчт',
       'ёша', 'ёшк', 'ёшь', 'ёэл', 'ігр', 'інк', 'ғаш', 'қжү', 'қна',
       'үре'], dtype=object)

In [ ]:
vect_bigram_df.toarray().shape

(15963, 13387)

## a. KNN slkearn brute

In [ ]:
X, y = vect_bigram_df.toarray(), np.array(names_df['item_id'].to_list())

knn = KNeighborsClassifier(n_neighbors=3, algorithm='brute', metric='l2')
knn.fit(X, y)

KNeighborsClassifier(algorithm='brute', metric='l2', n_neighbors=3)

In [ ]:
def search_by_knn_3grams(query):
    test_name_vec = vectorizer_cnt.transform(
        [' '.join(list(map(''.join, list(ngrams(re.sub(r'\W+', '', query), 3)))))]
    )

    out_df = names_df[names_df['item_id'] == knn.predict(test_name_vec.toarray())[0]]

    return out_df

In [ ]:
search_by_knn_3grams('аднажды в голивуде')

,item_id,title,title_orig,release_year,search_txt,token
6542,1015,[4k] Однажды в… Голливуде,Once Upon A Time In Hollywood,2019,4kоднаждывголливуде,4kо kод одн дна наж ажд жды дыв ывг вго гол ол...


In [ ]:
search_by_knn_3grams('сумерки')

,item_id,title,title_orig,release_year,search_txt,token
639,7280,Бумер,Bumer,2003,бумер,бум уме мер


In [ ]:
search_by_knn_3grams('шрек')

,item_id,title,title_orig,release_year,search_txt,token
535,3943,Ч/Б,Ch/B,2014,чб,


In [ ]:
search_by_knn_3grams('45 лет')

,item_id,title,title_orig,release_year,search_txt,token
535,3943,Ч/Б,Ch/B,2014,чб,


In [ ]:
search_by_knn_3grams('Лунтик и его друзья')

,item_id,title,title_orig,release_year,search_txt,token
11907,5294,Друзья,Friends,1994,друзья,дру руз узь зья


In [ ]:
nkcsnlcd = search_by_knn_3grams('Лунтик')[['item_id', 'title']]

In [ ]:
tuple(nkcsnlcd.to_numpy()[0])

(3943, 'Ч/Б')

In [ ]:
def apply_search_func(x, search_func, field):
    try:
        result = search_func(x)[field].to_list()[0]
    except IndexError:
        result = np.NaN

    return result

def assess_by_sample(search_func, sample_df):
    assess_df = sample_df.copy()
    assess_df['found_item_id'] = assess_df['query'].apply(
        lambda x: apply_search_func(x, search_func, 'item_id')
        )
    assess_df['found_title'] = assess_df['query'].apply(
        lambda x: apply_search_func(x, search_func, 'title')
        )

    assess_df['is_right'] = (assess_df['item_id'] == assess_df['found_item_id'])
    accuracy_score = round(sum(assess_df['is_right']) / len(assess_df), 3)

    return assess_df, accuracy_score

In [ ]:
assessment_df_knn_sk, score_knn_sk = assess_by_sample(
    search_by_knn_3grams, assessment_df
    )

In [ ]:
assessment_df_knn_sk

,item_id,title,query,found_item_id,found_title,is_right
0,10021,История искусств вместе с Хрюшей и...,история искусств,3943,Ч/Б,False
1,10021,История искусств вместе с Хрюшей и...,история искусств вместе,6806,Не вместе,False
2,10021,История искусств вместе с Хрюшей и...,история искусств c Хрюшей,6251,Темная история,False
3,10021,История искусств вместе с Хрюшей и...,история искуств c хрюший,3943,Ч/Б,False
4,14274,Playmobil Фильм: Через вселенные,playmobil,3943,Ч/Б,False
5,14274,Playmobil Фильм: Через вселенные,через вселенные,2255,Пленный,False
6,4458,[4k] Таиланд,таиланд,3943,Ч/Б,False
7,10149,Пунта Саль и обезьяны. Гондурас,гондурас обезьяны,246,Обезьянки,False
8,5755,Области тьмы,области тьмы,1696,Пластик,False
9,5755,Области тьмы,облости тьмы,4992,Гости,False


In [ ]:
score_knn_sk

0.217

## b. faiss

In [ ]:
dim = vect_bigram_df.toarray().shape[1]
dim

13387

In [ ]:
y_true_np = names_df['item_id'].to_numpy()

In [ ]:
faiss_index_3gram = faiss.IndexFlatL2(dim)
print(faiss_index_3gram.ntotal)
faiss_index_3gram.add(vect_bigram_df.toarray())
print(faiss_index_3gram.ntotal)

0
15963


In [ ]:
def search_by_faiss_3grams(query):
    test_name_vec = vectorizer_cnt.transform(
        [' '.join(list(map(''.join, list(ngrams(re.sub(r'\W+', '', query), 3)))))]
    )

    out_df = names_df[names_df['item_id'] == faiss_index_3gram.search(
        test_name_vec.toarray(), 1
        )[1][0][0]]

    return out_df

In [ ]:
search_by_faiss_3grams('аднажды в голивуде')

,item_id,title,title_orig,release_year,search_txt,token
14998,5605,Адский смерч,Fire Twister,2015,адскийсмерч,адс дск ски кий ийс йсм сме мер ерч


In [ ]:
assessment_df_faiss_ngram, score_faiss_ngram = assess_by_sample(
    search_by_faiss_3grams, assessment_df
    )

In [ ]:
score_faiss_ngram

0.0

In [ ]:
assessment_df_faiss_ngram

## c. MinHash LSH

In [ ]:
# Create LSH index
lsh = MinHashLSH(threshold=0.6, num_perm=128)

for idx in range(len(names_df['token'])):
    item_id = names_df['item_id'][idx]
    set_token = set(names_df['token'][idx].split(' '))
    mh = MinHash(num_perm=128)
    for el in set_token:
        mh.update(el.encode('utf8'))

    lsh.insert(item_id, mh)

In [ ]:
def search_in_lsh_index(query):
    test_name_lsh_set = set(
        map(''.join, list(ngrams(re.sub(r'\W+', '', query), 3)))
        )
    test_mh = MinHash(num_perm=128)
    for el in test_name_lsh_set:
        test_mh.update(el.encode('utf8'))

    result = lsh.query(test_mh)

    return names_df[names_df['item_id'].isin(result)]

In [ ]:
search_in_lsh_index('аднажды в голивуде')

,item_id,title,title_orig,release_year,search_txt,token
5605,6417,Однажды в Голливуде,What Just Happened,2008,однаждывголливуде,одн дна наж ажд жды дыв ывг вго гол олл лли ли...
6542,1015,[4k] Однажды в… Голливуде,Once Upon A Time In Hollywood,2019,4kоднаждывголливуде,4kо kод одн дна наж ажд жды дыв ывг вго гол ол...
7316,1877,Однажды в… Голливуде,Once Upon a Time... in Hollywood,2019,однаждывголливуде,одн дна наж ажд жды дыв ывг вго гол олл лли ли...


In [ ]:
search_in_lsh_index('сумерк')

,item_id,title,title_orig,release_year,search_txt,token
12925,14685,Сумерки,Twilight,2008,сумерки,сум уме мер ерк рки


In [ ]:
search_in_lsh_index('шрек')

,item_id,title,title_orig,release_year,search_txt,token
10557,13463,Шрек 2,Shrek 2,2004,шрек2,шре рек ек2
13429,14215,Шрек,Shrek,2001,шрек,шре рек


In [ ]:
search_in_lsh_index('45 лет')

,item_id,title,title_orig,release_year,search_txt,token
3,7868,45 лет,45 Years,2015,45лет,45л 5ле лет


In [ ]:
search_in_lsh_index('Лунтик')

,item_id,title,title_orig,release_year,search_txt,token


In [ ]:
search_in_lsh_index('Лунтик и его друзья')

,item_id,title,title_orig,release_year,search_txt,token
5585,8101,Лунтик и его друзья,Luntik I ego druzya,2006,лунтикиегодрузья,лун унт нти тик ики кие иег его год одр дру ру...


In [ ]:
assessment_df_lsh, score_lsh = assess_by_sample(
    search_in_lsh_index, assessment_df
    )

In [ ]:
score_lsh

0.467

In [ ]:
assessment_df_lsh

,item_id,title,query,found_item_id,found_title,is_right
0,10021,История искусств вместе с Хрюшей и...,история искусств,NaN,NaN,False
1,10021,История искусств вместе с Хрюшей и...,история искусств вместе,10021.0,История искусств вместе с Хрюшей и...,True
2,10021,История искусств вместе с Хрюшей и...,история искусств c Хрюшей,10021.0,История искусств вместе с Хрюшей и...,True
3,10021,История искусств вместе с Хрюшей и...,история искуств c хрюший,NaN,NaN,False
4,14274,Playmobil Фильм: Через вселенные,playmobil,NaN,NaN,False
5,14274,Playmobil Фильм: Через вселенные,через вселенные,12908.0,Человек-паук: Через вселенные,False
6,4458,[4k] Таиланд,таиланд,4458.0,[4k] Таиланд,True
7,10149,Пунта Саль и обезьяны. Гондурас,гондурас обезьяны,NaN,NaN,False
8,5755,Области тьмы,области тьмы,5755.0,Области тьмы,True
9,5755,Области тьмы,облости тьмы,NaN,NaN,False


__Проблема LSH:__
- При высоком пороге не покажет название фильма, если введена только его часть
- При низком пороге покажет слишком много фильмов, а отранжировать их по Джакару не понимаю как - в идеале ранжирование по Джакару и популярности в этом методе + можно попробовать взвешенный minhash (но он не сильно сыграет роль, так как вектора будут преимущественно бинарными)

# 2. Модель TFIDF и BM25

In [ ]:
docs_for_tfidf = [
    names_df['token'].to_list()
]

tfidf_vectorizer = TfidfVectorizer()

# Применение TF-IDF к текстовым данным
tfidf_matrix = tfidf_vectorizer.fit_transform(docs_for_tfidf[0])

# Получение списка ключевых слов и их значения TF-IDF для первого документа
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix.toarray()[0]

In [ ]:
tfidf_matrix.toarray().shape

(15963, 13387)

In [ ]:
dim_tfidf = tfidf_matrix.toarray().shape[1]
dim_tfidf

13387

In [ ]:
y_true_np = names_df['item_id'].to_numpy()

In [ ]:
faiss_index_tfidf = faiss.IndexFlatL2(dim_tfidf)
print(faiss_index_tfidf.ntotal)
faiss_index_tfidf.add(tfidf_matrix.toarray())
print(faiss_index_tfidf.ntotal)

0
15963


In [ ]:
def search_by_faiss_tfidf(query):
    test_name_vec = tfidf_vectorizer.transform(
        [query]
    )

    out_df = names_df[names_df['item_id'] == faiss_index_tfidf.search(
        test_name_vec.toarray(), 1
        )[1][0][0]]

    return out_df

In [ ]:
search_by_faiss_tfidf('однажды в голливуде')

,item_id,title,title_orig,release_year,search_txt,token
10587,535,Печенье с предсказанием,,2015,печеньеспредсказанием,печ ече чен ень нье ьес есп спр пре ред едс дс...


In [ ]:
assessment_df_faiss_tfidf, score_faiss_tfidf = assess_by_sample(
    search_by_faiss_tfidf, assessment_df
    )

In [ ]:
score_faiss_tfidf

0.0

In [ ]:
assessment_df_faiss_tfidf

# 3. Модель на Word2Vec

In [ ]:
# Show all available models in gensim-data
print(list(gs_downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
rus_keyed_vectors_pretrained = gs_downloader.load('word2vec-ruscorpora-300')

[==================================================] 100.0% 198.8/198.8MB downloaded


In [ ]:
rus_keyed_vectors_pretrained.most_similar('президент_NOUN')

[('путин_NOUN', 0.772294819355011),
 ('президентский_ADJ', 0.7611932754516602),
 ('ельцин_NOUN', 0.701576828956604),
 ('премьер-министр_NOUN', 0.6987968683242798),
 ('татарстан::минтимер_NOUN', 0.6845797300338745),
 ('первез::мушарраф_NOUN', 0.6816185116767883),
 ('вице-президент_NOUN', 0.6803255081176758),
 ('казахстан::нурсултан_NOUN', 0.6660419702529907),
 ('ваджпай_NOUN', 0.6552684903144836),
 ('аскар::акаев_NOUN', 0.6514782905578613)]

In [ ]:
rus_keyed_vectors_pretrained['президент_NOUN'].shape

(300,)

In [ ]:
len(rus_keyed_vectors_pretrained.key_to_index)

184973

In [ ]:
# Download the "glove-twitter-25" embeddings
glove_vectors = gs_downloader.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
glove_vectors.most_similar('однажды')

[('вдруг', 0.8865536451339722),
 ('думает', 0.8705506324768066),
 ('когда-то', 0.8613543510437012),
 ('скажет', 0.8604816198348999),
 ('сне', 0.8572654724121094),
 ('казалось', 0.8567517995834351),
 ('помните', 0.8567044138908386),
 ('говорил', 0.8525136709213257),
 ('думал', 0.8512946367263794),
 ('думаешь', 0.8508177399635315)]

In [ ]:
glove_vectors.most_similar('вода')

[('жопа', 0.9031836986541748),
 ('земля', 0.8994431495666504),
 ('холодная', 0.8907572031021118),
 ('белая', 0.8891905546188354),
 ('река', 0.8869621157646179),
 ('водка', 0.8825559616088867),
 ('луна', 0.881720781326294),
 ('течет', 0.8683515787124634),
 ('кровь', 0.8637696504592896),
 ('еда', 0.8630508184432983)]

In [ ]:
glove_vectors['президент'].shape

(25,)

In [ ]:
glove_vectors.cosine_similarities(
    vector_1=glove_vectors['президент'], vectors_all=[glove_vectors['руководитель'], glove_vectors['менеджер']]
    )

array([0.69613105, 0.59172624], dtype=float32)

In [ ]:
len(glove_vectors.key_to_index)

1193514

In [ ]:
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()

similarity = glove_vectors.wmdistance(sentence_obama, sentence_president)
print(f"{similarity:.4f}")

0.4657


In [ ]:
# documents - это список ваших документов, где каждый документ - это список слов

def document_vector(model, document):
    """Преобразует документ в вектор, усредняя векторы его слов."""
    return np.mean([model.wv[word] for word in document], axis=0)

def document_similarity(model, document, documents_array):
    """Расчитывает косинусное сходство между документом и массивом документов."""
    document_vec = document_vector(model, document)
    similarities = [model.wv.similarity(document_vec, document_vector(model, doc)) for doc in documents_array]
    return similarities

# Пример использования
similarities = document_similarity(w2v_model, my_document, documents_array)